# Figures for manuscript

## Figure List
1. BrainSense Artifact Creation
2. Sync Artifact Comparison
3. Spikedrop
4. Manual annotations
5. Tapping
6. Pipeline

7. 0.1 reduction
8. robustness of 2 artifacts

In [3]:
import sys
import os

# Get the path to the project root (one level up from the notebook)
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Add the project root, NOT 'source', to sys.path
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# print the sys.path
print(sys.path)

['/opt/homebrew/Cellar/python@3.12/3.12.3/Frameworks/Python.framework/Versions/3.12/lib/python312.zip', '/opt/homebrew/Cellar/python@3.12/3.12.3/Frameworks/Python.framework/Versions/3.12/lib/python3.12', '/opt/homebrew/Cellar/python@3.12/3.12.3/Frameworks/Python.framework/Versions/3.12/lib/python3.12/lib-dynload', '', '/Users/lenasalzmann/dev/dbs-eeg-sync/.venv/lib/python3.12/site-packages', '/Users/lenasalzmann/dev/dbs-eeg-sync']


In [4]:
import numpy as np
import pandas as pd
import mne
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter, resample_poly
import plotly.graph_objects as go
from dbs_eeg_sync.data_loader import load_eeg_data, open_json_file, select_recording, read_time_domain_data
from dbs_eeg_sync.sync_artifact_finder import detect_eeg_sync_artifact, detect_dbs_sync_artifact
from dbs_eeg_sync.power_calculator import compute_samplewise_eeg_power
from dbs_eeg_sync.plotting import apply_publication_style, EEG_COLOR, DBS_COLOR
from dbs_eeg_sync.synchronizer import cut_data_at_sync, synchronize_data


Use P4-2004 for EEG and P4-2001 for D and E

In [5]:
sub_ids = [ 'P4-2001', 'P4-2002', 'P4-2003', 'P4-2004', 'P4-2005', 'P4-2007', 'P4-2008', 'P4-2009', 'P4-2010', 'P4-2011']
# sub_id = 'P4-2001'
sub_id = 'P4-2004'
# sub_id = 'P4-2008'
block = 'baseline'
dataDir_server = r"/Volumes/03_Neurofeedback/project_only/02_Data/Patient_NFB"

In [6]:
file_info_dbs = [
    {"sub_id": "P4-2001", "file_path": "/Volumes/03_Neurofeedback/project_only/02_Data/Patient_NFB/P4-2001/RawData/DBS/Report_Json_Session_Report_20240610T162159.json"},
    {"sub_id": "P4-2002", "file_path": "/Volumes/03_Neurofeedback/project_only/02_Data/Patient_NFB/P4-2002/RawData/DBS/Report_Json_Session_Report_20240719T121230.json"},
    {"sub_id": "P4-2003", "file_path": "/Volumes/03_Neurofeedback/project_only/02_Data/Patient_NFB/P4-2003/RawData/DBS/Report_Json_Session_Report_20241025T120701.json"},
    {"sub_id": "P4-2004", "file_path": "/Volumes/03_Neurofeedback/project_only/02_Data/Patient_NFB/P4-2004/RawData/DBS/Report_Json_Session_Report_20241028T163753.json"},
    {"sub_id": "P4-2005", "file_path": "/Volumes/03_Neurofeedback/project_only/02_Data/Patient_NFB/P4-2005/RawData/DBS/Report_Json_Session_Report_20241029T170455.json"},
    {"sub_id": "P4-2007", "file_path": "/Volumes/03_Neurofeedback/project_only/02_Data/Patient_NFB/P4-2007/RawData/DBS/Report_Json_Session_Report_20250110T112532.json"},
    {"sub_id": "P4-2008", "file_path": "/Volumes/03_Neurofeedback/project_only/02_Data/Patient_NFB/P4-2008/RawData/DBS/Report_Json_Session_Report_20250310T113259.json"},
    {"sub_id": "P4-2009", "file_path": "/Volumes/03_Neurofeedback/project_only/02_Data/Patient_NFB/P4-2009/RawData/DBS/Report_Json_Session_Report_20250314T131338.json"},
    {"sub_id": "P4-2010", "file_path": "/Volumes/03_Neurofeedback/project_only/02_Data/Patient_NFB/P4-2010/RawData/DBS/Report_Json_Session_Report_20250425T161748.json"},
    {"sub_id": "P4-2011", "file_path": "/Volumes/03_Neurofeedback/project_only/02_Data/Patient_NFB/P4-2011/RawData/DBS/Report_Json_Session_Report_20250520T165018.json"},
    {"sub_id": "P4-2012", "file_path": "/Volumes/03_Neurofeedback/project_only/02_Data/Patient_NFB/P4-2012/RawData/DBS/Report_Json_Session_Report_20250710T122046.json"},
]

Load Data 

In [7]:
# Load EEG data
eeg_file = os.path.join(dataDir_server, sub_id, 'ProcessedData', 'EEG', block, block + '_prep.set')
eeg_data, sfreq = load_eeg_data(eeg_file)

In [8]:
# Load DBS

# get file_path from file_info_dbs
if block == 'baseline':
    block_number = 0
# add block indexing later
dbs_path = [file for file in file_info_dbs if file['sub_id'] == sub_id][block_number]['file_path']

json_data = open_json_file(dbs_path)
dbs_data = read_time_domain_data(json_data, block_number) 
dbs_signal =  dbs_data["TimeDomainData"].values
dbs_fs = dbs_data["SampleRateInHz"][0]

In [9]:
# print the length of the original recordings for eeg and dbs
print(f'Length of original eeg: {len(eeg_data.get_data()[0])/sfreq} seconds')
print(f'Length of original dbs: {len(dbs_data["TimeDomainData"].values)/dbs_fs} seconds')

# and in samples
print(f'Length of original eeg: {len(eeg_data.get_data()[0])} samples')
print(f'Length of original dbs: {len(dbs_data["TimeDomainData"].values)} samples')


Length of original eeg: 146.1675 seconds
Length of original dbs: 142.0 seconds
Length of original eeg: 292335 samples
Length of original dbs: 35500 samples


### 0. Overview: Run Sync Detection

EEG

In [10]:
# EEG
if sub_id == 'P4-2007':
    channel, eeg_sync_idx, eeg_sync_s, result, smoothed_power = detect_eeg_sync_artifact(eeg_data, freq_low=110, freq_high=120, plot=False, save_dir=None, sub_id=sub_id, block=block)
else:
    channel, eeg_sync_idx, eeg_sync_s, result, smoothed_power = detect_eeg_sync_artifact(eeg_data, freq_low=120, freq_high=130, plot=False, save_dir=None, sub_id=sub_id, block=block)

In [11]:
# print length of smoothed_power
print(f'Length of smoothed_power: {len(smoothed_power)} samples')
print(f'Length of smoothed_power: {len(smoothed_power)/sfreq} seconds')

Length of smoothed_power: 292335 samples
Length of smoothed_power: 146.1675 seconds


In [12]:
# extract the smoothed_power samples around +- 0.5s around the artifact for the plot
eeg_power_artifact = smoothed_power[eeg_sync_idx-int(eeg_data.info['sfreq']/2):eeg_sync_idx+int(eeg_data.info['sfreq']/2)]
eeg_times_artifact = np.arange(len(eeg_power_artifact)) / eeg_data.info['sfreq']

In [13]:
# check the downsampling of the sync index
print(f'Sync index: {eeg_sync_idx} samples')
print(f'Sync index: {eeg_sync_idx/eeg_data.info["sfreq"]} seconds')


Sync index: 12129 samples
Sync index: 6.0645 seconds


DBS

In [14]:
# Find DBS sync artifact
dbs_peak_idx, dbs_peak_s = detect_dbs_sync_artifact(dbs_signal, dbs_fs, save_dir=None, sub_id=sub_id, block=block)
# investigate the artifact
dbs_artifact = dbs_signal[dbs_peak_idx-int(dbs_fs/2):dbs_peak_idx+int(dbs_fs/2)]
dbs_times = np.arange(len(dbs_artifact)) / dbs_fs

### 1. Plot the first couple samples of the raw signals 

EEG

In [15]:
# select the following channels from the eeg data

# eeg_data_ch_selected = eeg_data.copy().pick_channels(['Pz', 'Cz', 'Fz', 'POz', 'CPz'])
eeg_data_ch_selected = eeg_data.copy().pick_channels(['Cz', 'Fz', 'POz'])

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


Fig. 6a - removed in final version

In [16]:
# extract the first 30 seconds of the eeg data for 5 channels

fs_eeg_org = int(eeg_data_ch_selected.info['sfreq'])
n_samples = int(0.38 * fs_eeg_org)  # 0.38 gives the correct length of the signal for the publication figure

channel_data, times = eeg_data_ch_selected[:5, 1:n_samples]   

eeg_sample_indices = np.arange(channel_data.shape[1])

n_channels = channel_data.shape[0]

fig_height_per_channel = 1

fig, axes = plt.subplots(n_channels + 1, 1, figsize=(5, fig_height_per_channel * (n_channels + 1)), sharex=True)

# Plot EEG channels

scale_factor = 0.8

for i in range(n_channels):
    # current_channel = channel_data[i]*1e6   # convert to uV
    current_channel = channel_data[i]
    ax = axes[i]
    ax.plot(eeg_sample_indices, current_channel, color=EEG_COLOR)  # sample indices

    ax.set_ylabel(eeg_data_ch_selected.ch_names[i], rotation=0, labelpad=1, fontsize=14, ha="right")
    ax.set_yticks([])
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_visible(False)
    if i != n_channels:
        ax.spines['bottom'].set_visible(False)
    ax.tick_params(axis='x', labelsize=14)
    ax.grid(True, linestyle="--", alpha=0.3)

# Empty subplot
axes[-1].set_ylabel("⋯", rotation=0, labelpad=1, fontsize=14, ha="center")
axes[-1].set_yticks([])
axes[-1].spines['top'].set_visible(False)
axes[-1].spines['right'].set_visible(False)
axes[-1].spines['left'].set_visible(False)
axes[-1].spines['bottom'].set_visible(True)
axes[-1].tick_params(axis='x', labelsize=14)
axes[-1].grid(True, linestyle="--", alpha=0.3)  # optional grid

# Labels
# axes[-1].set_xlabel("Time (s)", fontsize=14)
axes[-1].set_xlabel("Samples", fontsize=14)
fig.suptitle("Raw EEG Data", fontsize=16)

plt.tight_layout()
fig.text(-0.01, 0.55, "EEG Channel Data (μV)", va='center', rotation='vertical', fontsize=14)  # Shared y-label

# Add capital letter A to top left
fig.text(0.02, 0.98, 'A', transform=fig.transFigure, fontsize=18, fontweight='bold', va='top', ha='left')

# save plot as pdf in outputs/plots
# plt.savefig(f"outputs/plots/fig6a_raw_eeg.pdf", bbox_inches='tight')

plt.show()

DBS

In [17]:
# select the whole dbs data

dbs_selected_data = dbs_data["TimeDomainData"].values

# cut first seconds of dbs data
dbs_n_samples = int(3 * dbs_fs)  # 250 sampling rate, so 3 seconds account for 750 samples
dbs_selected_data = dbs_data["TimeDomainData"].values[1:dbs_n_samples]  # first seconds of recording

# arrange dbs_times over the length of dbs_selected_data
dbs_times = np.linspace(0, (len(dbs_selected_data)/dbs_fs), len(dbs_selected_data))

dbs_sample_indices = np.arange(len(dbs_selected_data))

In [18]:
from scipy.signal import butter, filtfilt

def bandpass_filter(data, fs=250.0, lowcut=1.0, highcut=100.0, order=4):
    """
    Apply a zero-phase Butterworth bandpass filter (filtfilt) to EEG/DBS data.
    
    Parameters
    ----------
    data : array_like
        1D signal (time-domain).
    fs : float
        Sampling frequency in Hz.
    lowcut : float
        Low cutoff frequency in Hz.
    highcut : float
        High cutoff frequency in Hz.
    order : int
        Filter order (default=4).
    
    Returns
    -------
    filtered : ndarray
        Bandpass-filtered signal.
    """
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    return filtfilt(b, a, data)

In [19]:
# filter the dbs signal from 1-100 Hz
dbs_selected_data_filt = bandpass_filter(dbs_selected_data, fs=dbs_fs, lowcut=1, highcut=100)

Fig. 5c

In [20]:
fig, ax = plt.subplots(figsize=(6, 3))

# ax.plot(dbs_times, dbs_selected_data, color=DBS_COLOR)
ax.plot(dbs_sample_indices, dbs_selected_data, color=DBS_COLOR)
# ax.plot(dbs_sample_indices, dbs_selected_data_filt, color=DBS_COLOR)
ax.set_ylabel('LFP (μV)', rotation=90, labelpad=1, fontsize=18, ha="center")

# Remove y-ticks
ax.set_yticks([])

# xticks fontsize 14
ax.tick_params(axis='x', labelsize=18)

# Hide all spines except bottom
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.grid(True, linestyle="--", alpha=0.3)  # optional grid

# ax.set_xlabel("Time (s)")
ax.set_xlabel("Samples", fontsize=18)
fig.suptitle("Raw DBS LFP Data", fontsize=18)

# add capital letter D to top left
fig.text(0.02, 0.98, 'C', transform=fig.transFigure, fontsize=18, fontweight='bold', va='top', ha='left')

plt.tight_layout()

# save plot as pdf in outputs/plots
if sub_id == "P4-2001":
    plt.savefig(f"outputs/plots/fig6c_raw_dbs.pdf")
plt.show()


### 2. EEG Power Calculation

In [21]:
# Power for ['Pz', 'Cz', 'Fz', 'POz', 'CPz'] of complete signal (eeg_data_ch_selected)

# _, _, _, _, power_Pz = detect_eeg_sync_artifact(eeg_data_ch_selected, freq_low=120, freq_high=130, channel_list=['Pz'], plot=False, save_dir=None, sub_id=sub_id, block=block)
_, _, _, _, power_Cz = detect_eeg_sync_artifact(eeg_data_ch_selected, freq_low=120, freq_high=130, channel_list=['Cz'], plot=False, save_dir=None, sub_id=sub_id, block=block)
_, _, _, _, power_Fz = detect_eeg_sync_artifact(eeg_data_ch_selected, freq_low=120, freq_high=130, channel_list=['Fz'], plot=False, save_dir=None, sub_id=sub_id, block=block)
best_channel, eeg_sync_idx, eeg_sync_s, best_result,  power_post = detect_eeg_sync_artifact(eeg_data_ch_selected, freq_low=120, freq_high=130, channel_list=['POz'], plot=False, save_dir=None, sub_id=sub_id, block=block)
# _, _, _, _, power_CPz = detect_eeg_sync_artifact(eeg_data_ch_selected, freq_low=120, freq_high=130, channel_list=['CPz'], plot=False, save_dir=None, sub_id=sub_id, block=block)

# power_all = [power_Pz, power_Cz, power_Fz, power_post, power_CPz]

power_all = [power_Cz, power_Fz, power_post]
power_time = best_result["power_time"]
power_samples_indices = np.arange(len(power_time))

EEG power still sampled with 2000Hz

Fig. 5a

In [22]:
# plot the power for each channel

# Keep ~750 samples from the start
power_all = np.array(power_all)  # Convert list to array
power_all_cut = power_all[:, :750]
power_samples_indices = np.arange(power_all_cut.shape[1])

# Get number of channels
n_channels = power_all_cut.shape[0]

# Plot with one extra empty subplot
fig, axes = plt.subplots(n_channels + 1, 1, figsize=(6, n_channels + 1), sharex=True)

for i in range(n_channels):
    ax = axes[i]
    ax.plot(power_samples_indices, power_all_cut[i], color=EEG_COLOR)
    ax.set_ylabel(eeg_data_ch_selected.ch_names[i], rotation=0, labelpad=1, fontsize=18, ha="right")
    ax.set_yticks([])
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_visible(False)
    if i != n_channels:
        ax.spines['bottom'].set_visible(False)
    ax.grid(True, linestyle="--", alpha=0.3)

# Add empty subplot
axes[-1].set_ylabel("⋯", rotation=0, labelpad=1, fontsize=18, ha="center")
axes[-1].set_yticks([])
axes[-1].spines['top'].set_visible(False)
axes[-1].spines['right'].set_visible(False)
axes[-1].spines['left'].set_visible(False)
axes[-1].spines['bottom'].set_visible(True)
axes[-1].grid(linestyle="--", alpha=0.3)

axes[-1].tick_params(axis='x', labelsize=14)
# set xticks to 0, 200, 400, 600
axes[-1].set_xticks([0, 200, 400, 600])
axes[-1].set_xlabel("Samples", fontsize=14)
fig.suptitle("Filtered and Smoothed EEG Power", fontsize=18)
plt.tight_layout()

# Shared y-label
fig.text(-0.01, 0.55, "EEG Power (μV²/Hz)", va='center', rotation='vertical', fontsize=18)

# Add capital letter B to top left
fig.text(0.02, 0.98, 'A', transform=fig.transFigure, fontsize=18, fontweight='bold', va='top', ha='left')

# save plot as pdf in outputs/plots
if sub_id == "P4-2008":
    plt.savefig(f"outputs/plots/fig6a_eeg_power.pdf", bbox_inches='tight')
plt.show()


### 3. Artifact Selection

Fig. 5b - best for P4-2008

In [23]:
# plot eeg artifact with the sync artifact

best_power_smoothed = power_post

# Create time vector based on sampling frequency and start time
eeg_time_vector = np.arange(len(best_power_smoothed)) / fs_eeg_org + 0
eeg_sample_indices = np.arange(len(best_power_smoothed))

best_eeg_power_smoothed_cut = best_power_smoothed
eeg_power_time_vector_cut = eeg_time_vector
eeg_sample_indices_cut = eeg_sample_indices

#  cut the first samples and last samples as they have a big artifact
# best_eeg_power_smoothed_cut = best_power_smoothed[100:-100]
# eeg_power_time_vector_cut = eeg_time_vector[100:-100]
# eeg_sample_indices_cut = eeg_sample_indices[100:-100]

sync_time = best_result["onset_time"]
sync_idx = best_result["onset_index"]

plt.figure(figsize=(6, 3))
# Plot line
plt.plot(eeg_power_time_vector_cut, best_eeg_power_smoothed_cut, label=f"Power Channel {best_channel}", color=EEG_COLOR)
# plt.plot(eeg_sample_indices_cut, best_eeg_power_smoothed_cut, label="Smoothed Power", color=EEG_COLOR)  # for samples

# Sync artifact
# plt.axvline(sync_time, color="grey", linestyle="--", label=f"EEG Artifact")
plt.axvline(sync_time, color="grey", linestyle="--", linewidth=1.5)
# plt.axvline(sync_idx, color="grey", linestyle="--", label=f"EEG Artifact: {sync_idx} samples")

# Styling
plt.ylabel("EEG Power (μV²/Hz)", fontsize=14)
plt.xlabel("Time (s)", fontsize=14)  # change to "Samples" if using index
plt.title(f"EEG Artifact Detection", fontsize=18)

# add an arrow at the detected artifact in the upper third, pointing horizontally
y_max = best_eeg_power_smoothed_cut.max()
y_min = best_eeg_power_smoothed_cut.min()
y_range = y_max - y_min
x_range = eeg_power_time_vector_cut[-1] - eeg_power_time_vector_cut[0]  # total time range

# Position arrow to the right of the artifact
x_offset = 0.1 * x_range  # Move arrow start 10% of x-range to the right
arrow_x_start = sync_time + x_offset
arrow_y_start = y_min + 0.3 * y_range  # Position at 0.3% of the height

# Arrow pointing horizontally left towards the artifact
arrow_tip_x = sync_time  # Arrow points to the artifact line

# Use annotate for better arrow rendering
plt.annotate('EEG Artifact', 
             xy=(arrow_tip_x, arrow_y_start),  # tip of arrow
             xytext=(arrow_x_start + 0.02*x_range, arrow_y_start),  # text position
             fontsize=16, color='grey', ha='left', va='center',
             arrowprops=dict(arrowstyle='->', color='grey', lw=1.5))

# Hide spines except bottom
for spine in ['top', 'right', 'left']:
    plt.gca().spines[spine].set_visible(False)

# yticks empty
plt.yticks([])
# fontsize 14
plt.tick_params(axis='x', labelsize=14)

plt.grid(True, linestyle="--", alpha=0.3)
plt.legend(fontsize=12, loc='upper right')

# Add capital letter E to top left
plt.gcf().text(0.02, 0.98, 'B', transform=plt.gcf().transFigure, fontsize=18, fontweight='bold', va='top', ha='left')


plt.tight_layout()
# save plot as pdf in outputs/plots
if sub_id == "P4-2008":
    plt.savefig(f"outputs/plots/fig6b_eeg_power_artifact.pdf")
plt.show()

Fig. 5d

In [24]:
# Find DBS sync artifact
dbs_signal =  dbs_data["TimeDomainData"].values

# filter the dbs signal from 1-100 Hz
dbs_signal = bandpass_filter(dbs_signal, fs=dbs_fs, lowcut=1, highcut=100)


dbs_fs = dbs_data["SampleRateInHz"][0]
dbs_peak_idx, dbs_peak_s = detect_dbs_sync_artifact(dbs_signal, dbs_fs, save_dir=None, sub_id=sub_id, block=block)

# plot again in publication style
dbs_time_axis = np.arange(len(dbs_signal)) / dbs_fs

plt.figure(figsize=(6, 3))

# Plot DBS signal
plt.plot(dbs_time_axis, dbs_signal, label="DBS LFP", color=DBS_COLOR)
# plt.plot(np.arange(len(dbs_signal)), dbs_signal, label="DBS Signal", color=DBS_COLOR)  # for samples

# Mark detected artifact
# plt.axvline(dbs_time_axis[dbs_peak_idx], color='grey', linestyle='--', label=f'DBS Artifact')
# plt.axvline(dbs_peak_idx, color='grey', linestyle='--', label=f'DBS Artifact: {dbs_peak_idx} samples')

# add an arrow at the detected artifact in the upper third, pointing horizontally
y_max = dbs_signal.max()
y_min = dbs_signal.min()
y_range = y_max - y_min
x_range = dbs_time_axis[-1] - dbs_time_axis[0]  # total time range

# Position arrow to the right of the artifact
x_offset = 0.2 * x_range  # Move arrow start 20% of x-range to the right
arrow_x_start = dbs_time_axis[dbs_peak_idx] + x_offset
arrow_y_start = y_min + 0.75 * y_range  # Position at 75% of the height

# Arrow pointing horizontally left towards the artifact
arrow_length = 0.15 * x_range  # Arrow length in x direction
arrow_dx = -arrow_length  # negative to point left
arrow_dy = 0  # horizontal arrow (no vertical component)

plt.arrow(arrow_x_start, arrow_y_start, arrow_dx, arrow_dy, 
          head_width=0.08*y_range, head_length=0.03*x_range, fc='grey', ec='grey')
# label the arrow with "artifact" to the right of the arrow
plt.text(arrow_x_start + 0.02*x_range, arrow_y_start, 
         'LFP Artifact', fontsize=18, color='grey', ha='left', va='center')

# Labels and style
plt.xlabel('Time (s)', fontsize=18)  # change to 'Samples' if using index
plt.ylabel('LFP (μV)', fontsize=18)
plt.title('LFP Artifact Detection', fontsize=18)

# Clean spines
for spine in ['top', 'right', 'left']:
    plt.gca().spines[spine].set_visible(False)

# xticks fontsize 14
plt.tick_params(axis='x', labelsize=14)

plt.yticks([])
plt.grid(True, linestyle="--", alpha=0.3)
plt.legend(fontsize=16, loc='upper right')

# Add capital letter E to top left
plt.gcf().text(0.02, 0.98, 'D', transform=plt.gcf().transFigure, fontsize=18, fontweight='bold', va='top', ha='left')


plt.tight_layout()
# save plot as pdf in outputs/plots
if sub_id == "P4-2008":
    plt.savefig(f"outputs/plots/fig6d_dbs_artifact.pdf")

plt.show()


### 4. Get samples around artifact and plot

Fig. 3a (spike)

In [25]:
# Drop
if sub_id == "P4-2001":
    # plot the signal around the artifact: 
    eeg_power_artifact_clean = best_power_smoothed[eeg_sync_idx - fs_eeg_org//2:eeg_sync_idx + fs_eeg_org//2]

    plt.figure(figsize=(5, 3))
    plt.plot(eeg_power_artifact_clean, color=EEG_COLOR)
    plt.axvline(1000, color='grey', linestyle='--', label=f'Synchronization Spike', linewidth=3)
    plt.ylabel('EEG Power (μV²/Hz)', fontsize=14)
    plt.xlabel('Samples', fontsize=14)
    plt.title('P01 - EEG channel Oz', fontsize=16)
    # add capital letter B to top left
    plt.gcf().text(0.02, 0.98, 'A', transform=plt.gcf().transFigure, fontsize=18, fontweight='bold', va='top', ha='left')
    plt.legend()
    plt.tight_layout()
    plt.savefig(f"outputs/plots/fig3a_eeg_power_artifact_spike.pdf")

    plt.show()

Fig. 3b (drop)


In [26]:
# Spike
if sub_id == "P4-2004":
    eeg_power_artifact_clean = best_power_smoothed[eeg_sync_idx - fs_eeg_org//2:eeg_sync_idx + fs_eeg_org//2]

    plt.figure(figsize=(5, 3))
    plt.plot(eeg_power_artifact_clean, color=EEG_COLOR)
    plt.axvline(1000, color='grey', linestyle='--', label=f'Synchronization Drop', linewidth=3)
    plt.ylabel('EEG Power (μV²/Hz)', fontsize=14)
    plt.xlabel('Samples', fontsize=14)
    plt.title('P04 - EEG channel Oz', fontsize=16)
    # add capital letter A to top left
    plt.gcf().text(0.02, 0.98, 'B', transform=plt.gcf().transFigure, fontsize=18, fontweight='bold', va='top', ha='left')
    plt.legend()
    plt.tight_layout()
    plt.savefig(f"outputs/plots/fig3b_eeg_power_artifact_drop.pdf")

    plt.show()
    

In [27]:
# first resample the eeg power to the dbs sampling rate

# Resample (downsample by factor 8)
eeg_power_resampled = resample_poly(best_power_smoothed, up=dbs_fs, down=fs_eeg_org)

In [28]:
# print the length of the downssampled eeg in seconds and samples
print(f'Length of downsampled eeg: {len(eeg_power_resampled)/dbs_fs} seconds')
print(f'Length of downsampled eeg: {len(eeg_power_resampled)} samples')

Length of downsampled eeg: 146.168 seconds
Length of downsampled eeg: 36542 samples


In [29]:
# resample the sync index
eeg_sync_idx = int(best_result["index"])
eeg_sync_idx_resampled = int(eeg_sync_idx * dbs_fs / fs_eeg_org)
print(f'Sync index downsampled: {eeg_sync_idx_resampled} samples')
print(f'Sync index downsampled: {eeg_sync_idx_resampled/dbs_fs} seconds')

Sync index downsampled: 1516 samples
Sync index downsampled: 6.064 seconds


In [30]:
# extract the smoothed_power samples around +- 0.5s around the resampled artifact

eeg_power_artifact = eeg_power_resampled[eeg_sync_idx_resampled - dbs_fs//2 : eeg_sync_idx_resampled + dbs_fs//2]
eeg_times_artifact = np.arange(len(eeg_power_artifact)) / dbs_fs

print(eeg_power_artifact.shape)
print(eeg_times_artifact.shape)

(250,)
(250,)


In [31]:
# get the DBS artifact
dbs_artifact = dbs_signal[dbs_peak_idx-int(dbs_fs/2):dbs_peak_idx+int(dbs_fs/2)]
dbs_times = np.linspace(0, 1.0, len(dbs_artifact))

Fig. 5e

In [32]:
fig, ax1 = plt.subplots(figsize=(8, 4))

# EEG power on primary y-axis
eeg_line, = ax1.plot(eeg_times_artifact, eeg_power_artifact, color=EEG_COLOR, label='EEG Power')
ax1.set_ylabel('EEG Power (μV²/Hz)', fontsize=18, color=EEG_COLOR)
ax1.set_xlabel('Time (s)', fontsize=18)
ax1.set_title(f'Aligned EEG and LFP Around Artifact', fontsize=18)

# DBS LFP on secondary y-axis
ax2 = ax1.twinx()
dbs_line, = ax2.plot(dbs_times, dbs_artifact, color=DBS_COLOR, label='DBS LFP')
ax2.set_ylabel('LFP (μV)', fontsize=18, color=DBS_COLOR)
ax2.set_yticks([])

# Vertical line for sync artifact, make it thicker
artifact_line = ax1.axvline(0.5, color='grey', linestyle='--', label='Detected artifact', linewidth=3)

# Styling: remove spines and y-ticks (left)
for ax in [ax1, ax2]:
    for spine in ['top', 'right', 'left']:
        ax.spines[spine].set_visible(False)

ax1.set_yticks([])
ax1.grid(True, linestyle="--", alpha=0.3)

# xticks fontsize 14
ax1.tick_params(axis='x', labelsize=18)

# Legend combining both axes
lines = [eeg_line, dbs_line, artifact_line]
labels = [line.get_label() for line in lines]
ax1.legend(lines, labels, fontsize=16, loc='upper right')

plt.tight_layout()

# Add capital letter F to top left
fig.text(0.02, 0.98, 'E', transform=fig.transFigure, fontsize=18, fontweight='bold', va='top', ha='left')

# save plot as pdf in outputs/plots
if sub_id == "P4-2004":
    plt.savefig(f"outputs/plots/fig6e_eeg_dbs_artifact.pdf")

plt.show()

### 5. Crop and align time series

In [33]:
# get eeg_data_ch_selected data for "best_channel"
data_2d, times = eeg_data_ch_selected.get_data(picks=[best_channel], return_times=True)
eeg_data_best_channel = data_2d[0]  # shape (n_times,)

In [34]:
# crop data at sync

cropped_eeg, cropped_dbs = cut_data_at_sync(
    eeg_raw=eeg_data_ch_selected,
    dbs_df=dbs_data,
    eeg_sync_idx=int(eeg_sync_idx),
    dbs_sync_idx=int(dbs_peak_idx)
)
# synchronize data
synchronized_eeg, synchronized_dbs = synchronize_data(
    cropped_eeg, cropped_dbs,
    resample_data=True,
    save_dir=None,          # or "outputs/plots" if you want the overlay figure saved
    sub_id=sub_id, block=block,
)

In [35]:
# get the best_channel from the synchronized_eeg
synchronized_eeg_1ch = synchronized_eeg.get_data(picks=[best_channel])[0]

In [36]:
# plot the synchronized data up until 120 seconds
# Use the minimum length to ensure both signals have the same number of samples
min_length = min(len(synchronized_eeg_1ch), len(synchronized_dbs["TimeDomainData"]), 120*dbs_fs)
synchronized_eeg_1ch_cropped = synchronized_eeg_1ch[0:min_length]
synchronized_dbs_cropped = synchronized_dbs["TimeDomainData"][0:min_length]
synced_eeg_times = np.arange(len(synchronized_eeg_1ch_cropped)) / dbs_fs

Fig. 5f

In [37]:
fig, ax1 = plt.subplots(figsize=(5, 3))

# EEG signal
eeg_line, = ax1.plot(synced_eeg_times, synchronized_eeg_1ch_cropped, color=EEG_COLOR, label='EEG')
ax1.set_ylabel('Raw EEG (μV)', fontsize=14, color=EEG_COLOR)
ax1.set_xlabel('Time (s)', fontsize=14)
ax1.set_title(f'EEG and LFP Time Series Aligned', fontsize=14)

# DBS signal
ax2 = ax1.twinx()
dbs_line, = ax2.plot(synced_eeg_times, synchronized_dbs_cropped, color=DBS_COLOR, label='LFP')
ax2.set_ylabel('LFP (μV)', fontsize=14, color=DBS_COLOR)

# Clean spines and ticks
for ax in [ax1, ax2]:
    for spine in ['top', 'right', 'left']:
        ax.spines[spine].set_visible(False)
    ax.set_yticks([])

ax1.grid(True, linestyle="--", alpha=0.3)

# xticks fontsize 14
ax1.tick_params(axis='x', labelsize=14)

# Combine legends
lines = [eeg_line, dbs_line]
labels = [line.get_label() for line in lines]
ax1.legend(lines, labels, loc='upper right', fontsize=12)

plt.tight_layout()

# Add capital letter G to top left
fig.text(0.02, 0.98, 'G', transform=fig.transFigure, fontsize=18, fontweight='bold', va='top', ha='left')

# save plot as pdf in outputs/plots
if sub_id == "P4-2008":
    plt.savefig(f"outputs/plots/fig6f_eeg_dbs.pdf")
plt.show()

### Channel Comparison: Oz vs AFz Artifact Amplitude


In [38]:
# print sub_id
print(sub_id)
# print available channels
print(eeg_data.ch_names)

P4-2004
['Fp1', 'Fp2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'O1', 'O2', 'F7', 'F8', 'T7', 'T8', 'P7', 'P8', 'AFz', 'Fz', 'Cz', 'Pz', 'FC1', 'FC2', 'CP1', 'CP2', 'FC5', 'FC6', 'CP5', 'CP6', 'FT9', 'FT10', 'TP7', 'TP8', 'F1', 'F2', 'C1', 'C2', 'P1', 'P2', 'AF3', 'AF4', 'FC3', 'FC4', 'CP3', 'CP4', 'PO3', 'PO4', 'F5', 'F6', 'C5', 'C6', 'P5', 'P6', 'AF7', 'AF8', 'FT7', 'FT8', 'TP9', 'TP10', 'PO7', 'PO8', 'PO9', 'PO10', 'CPz', 'POz']


In [39]:
# Select Oz and AFz channels from the full EEG data
# pick channels
ch_posterior = 'C1'
ch_frontal = 'F1'
eeg_data_post_front = eeg_data.copy().pick_channels([ch_posterior, ch_frontal])

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


In [40]:
# Compute power for Oz channel
if sub_id == 'P4-2007':
    channel_post, eeg_sync_idx_post, eeg_sync_s_post, result_post, power_post = detect_eeg_sync_artifact(
        eeg_data_post_front, freq_low=110, freq_high=120, channel_list=[ch_posterior], 
        plot=False, save_dir=None, sub_id=sub_id, block=block
    )
else:
    channel_post, eeg_sync_idx_post, eeg_sync_s_post, result_post, power_post = detect_eeg_sync_artifact(
        eeg_data_post_front, freq_low=120, freq_high=130, channel_list=[ch_posterior], 
        plot=False, save_dir=None, sub_id=sub_id, block=block
    )

# Compute power for AFz channel
if sub_id == 'P4-2007':
    channel_front, eeg_sync_idx_front, eeg_sync_s_front, result_front, power_front = detect_eeg_sync_artifact(
        eeg_data_post_front, freq_low=110, freq_high=120, channel_list=[ch_frontal], 
        plot=False, save_dir=None, sub_id=sub_id, block=block
    )
else:
    channel_front, eeg_sync_idx_front, eeg_sync_s_front, result_front, power_front = detect_eeg_sync_artifact(
        eeg_data_post_front, freq_low=120, freq_high=130, channel_list=[ch_frontal], 
        plot=False, save_dir=None, sub_id=sub_id, block=block
    )


In [41]:
# Extract artifact region for Oz channel (±0.5s around artifact)
eeg_power_artifact_post = power_post[eeg_sync_idx_post - int(sfreq/2) : eeg_sync_idx_post + int(sfreq/2)]
eeg_times_artifact_post = np.arange(len(eeg_power_artifact_post)) / sfreq

# Extract artifact region for AFz channel (±0.5s around artifact)
eeg_power_artifact_front = power_front[eeg_sync_idx_front - int(sfreq/2) : eeg_sync_idx_front + int(sfreq/2)]
eeg_times_artifact_front = np.arange(len(eeg_power_artifact_front)) / sfreq

print(f"{ch_posterior} artifact detected at: {eeg_sync_idx_post} samples ({eeg_sync_s_post:.3f} s)")
print(f"{ch_frontal} artifact detected at: {eeg_sync_idx_front} samples ({eeg_sync_s_front:.3f} s)")
print(f"{ch_posterior} peak artifact amplitude: {np.max(eeg_power_artifact_post):.2f}")
print(f"{ch_frontal} peak artifact amplitude: {np.max(eeg_power_artifact_front):.2f}")
print(f"Amplitude ratio ({ch_posterior}/{ch_frontal}): {np.max(eeg_power_artifact_post) / np.max(eeg_power_artifact_front):.2f}x")


C1 artifact detected at: 12137 samples (6.069 s)
F1 artifact detected at: 12091 samples (6.045 s)
C1 peak artifact amplitude: 0.00
F1 peak artifact amplitude: 0.00
Amplitude ratio (C1/F1): 4.18x


**Overlay plot**: Both channels on the same axes for direct comparison

In [42]:
EEG_COLOR
EEG_COLOR_light = '#E4929780'

Fig1: choose P4-2001 + baseline for publication plot with channel C1 and F1

**Stacked plot:** side-by-side with peak amplitude annotations

In [43]:
# Plot comparison: Oz vs AFz side by side
fig, axes = plt.subplots(2, 1, figsize=(7, 6), sharex=True)

# Oz channel (top)
axes[0].plot(eeg_times_artifact_post, eeg_power_artifact_post, color=EEG_COLOR, linewidth=2)
axes[0].axvline(0.5, color='grey', linestyle='--', linewidth=3, alpha=0.7)
axes[0].set_ylabel('EEG Power (μV²/Hz)', fontsize=14)
axes[0].set_title(f'{ch_posterior} (Central) - Near DBS Leads', fontsize=14, fontweight='bold')
axes[0].grid(True, linestyle="--", alpha=0.3)
axes[0].tick_params(axis='y', labelsize=12)

# # Add max amplitude annotation
# max_post = np.max(eeg_power_artifact_post)
# max_idx_post = np.argmax(eeg_power_artifact_post)
# axes[0].annotate(f'Peak: {max_post:.1f}', 
#                 xy=(eeg_times_artifact_post[max_idx_post], max_post),
#                 xytext=(10, 10), textcoords='offset points',
#                 fontsize=11, color=EEG_COLOR,
#                 bbox=dict(boxstyle='round,pad=0.3', facecolor='white', edgecolor=EEG_COLOR, alpha=0.8))

# AFz channel (bottom)
axes[1].plot(eeg_times_artifact_front, eeg_power_artifact_front, color=EEG_COLOR_light, linewidth=2)
axes[1].axvline(0.5, color='grey', linestyle='--', linewidth=3, alpha=0.7, label='Sync Artifact')
axes[1].set_ylabel('EEG Power (μV²/Hz)', fontsize=14)
axes[1].set_xlabel('Time (s)', fontsize=14)
axes[1].set_title(f'{ch_frontal} (Frontal) - Far from DBS Leads', fontsize=14, fontweight='bold')
axes[1].grid(True, linestyle="--", alpha=0.3)
axes[1].tick_params(axis='both', labelsize=12)

# # Add max amplitude annotation
# max_front = np.max(eeg_power_artifact_front)
# max_idx_front = np.argmax(eeg_power_artifact_front)
# axes[1].annotate(f'Peak: {max_front:.1f}', 
#                 xy=(eeg_times_artifact_front[max_idx_front], max_front),
#                 xytext=(10, 10), textcoords='offset points',
#                 fontsize=11, color=EEG_COLOR_light,
#                 bbox=dict(boxstyle='round,pad=0.3', facecolor='white', edgecolor=EEG_COLOR_light, alpha=0.8))

# Clean spines for both
for ax in axes:
    for spine in ['top', 'right']:
        ax.spines[spine].set_visible(False)

# Overall title
# fig.suptitle(f'Channel Comparison: Artifact Amplitude ({ch_posterior}/{ch_frontal} ratio: {max_post/max_front:.2f}x)',
fig.suptitle(f'Channel Comparison: Artifact Amplitude ({ch_posterior}, {ch_frontal})', 
             fontsize=16, fontweight='bold', y=0.995)

plt.tight_layout()
# plt.savefig(f"outputs/plots/fig_channel_comparison_post_vs_front_stacked.pdf", bbox_inches='tight')
plt.show()


In [44]:
# Labels and patient info
ch_posterior = 'C1'   # Central
ch_frontal   = 'F1'   # Frontal
patient_id   = 'P4-2001'

# Colors (replace with your palette if already defined)
try:
    EEG_COLOR
    EEG_COLOR_light
except NameError:
    EEG_COLOR = '#1f77b4'        # main line color
    EEG_COLOR_light = '#8fb9e2'  # lighter variant

# --- Plot ---
fig, ax = plt.subplots(figsize=(7, 3))

# Central (near leads)
line1, = ax.plot(eeg_times_artifact_post, eeg_power_artifact_post,
                 color=EEG_COLOR, linewidth=2.2, label='C1 (Central)')

# Frontal (far from leads)
line2, = ax.plot(eeg_times_artifact_front, eeg_power_artifact_front,
                 color=EEG_COLOR_light, linewidth=2.2, label='F1 (Frontal)')

# Sync artifact marker
line3 = ax.axvline(0.5, color='grey', linestyle='--', linewidth=3, alpha=0.7, label='Sync Artifact')

# Labels and style
ax.set_xlabel('Time (s)', fontsize=14)
ax.set_ylabel('EEG Power (μV²/Hz)', fontsize=14)
# ax.set_title(f'Artifact Comparison: {ch_posterior} vs {ch_frontal} EEG Channel, {patient_id}',
#              fontsize=16, fontweight='bold', pad=10)
ax.grid(True, linestyle='--', alpha=0.3)
ax.tick_params(axis='both', labelsize=12)

# Legend with box
legend = ax.legend(handles=[line1, line2, line3],
                   fontsize=11, loc='upper right',
                   frameon=True, fancybox=True,
                   framealpha=0.9, edgecolor='grey')

# Clean spines
for spine in ['top', 'right']:
    ax.spines[spine].set_visible(False)

plt.tight_layout()
if sub_id == "P4-2001":
    plt.savefig("outputs/plots/fig2_channel_comparison.pdf", bbox_inches='tight')
plt.show()

# Slider

In [45]:
# Configure matplotlib to use Qt backend for interactive GUI
%matplotlib qt

# This enables interactive Qt windows in Jupyter
# You may need to restart the kernel if you get backend errors

print("✓ Matplotlib backend set to Qt")
print("  You can now run interactive Qt GUIs from this notebook")


✓ Matplotlib backend set to Qt
  You can now run interactive Qt GUIs from this notebook


In [ ]:
# Test slider function with loaded data
# IMPORTANT: Reload the module to pick up any code changes
import importlib
import dbs_eeg_sync.gui
importlib.reload(dbs_eeg_sync.gui)
from dbs_eeg_sync.gui import manual_select_sync

# Use the already loaded data from cells above
# eeg_data: loaded in cell 8 (P4-2004)
# dbs_signal, dbs_fs: loaded in cell 9

print(f"Module reloaded successfully!")
print(f"Testing GUI with {sub_id} data")
print(f"EEG length: {len(eeg_data.get_data()[0])/sfreq:.1f}s, DBS length: {len(dbs_signal)/dbs_fs:.1f}s")

try:
    # This will open a Qt window with sliders
    result = manual_select_sync(
        eeg_data=eeg_data,           # MNE Raw object
        eeg_fs=sfreq,                # 2000 Hz
        dbs_data=dbs_signal,         # numpy array
        dbs_fs=dbs_fs,               # 250 Hz
        title=f"Manual Sync Test - {sub_id}",
        freq_low=120,
        freq_high=130,
        channel='POz'  # or any channel in your data
    )
    
    eeg_idx, eeg_t, dbs_idx, dbs_t = result
    print(f"\n✓ SUCCESS! Sliders work correctly!")
    print(f"  EEG selected: idx={eeg_idx}, t={eeg_t:.3f}s")
    print(f"  DBS selected: idx={dbs_idx}, t={dbs_t:.3f}s")
    
except RuntimeError as e:
    print(f"\n✗ Cancelled or error: {e}")
except Exception as e:
    print(f"\n✗ Unexpected error: {e}")
    import traceback
    traceback.print_exc()


Module reloaded successfully!
Testing GUI with P4-2004 data
EEG length: 146.2s, DBS length: 142.0s


/Users/lenasalzmann/dev/dbs-eeg-sync/dbs_eeg_sync/gui.py:410: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  pdf.savefig(fig, bbox_inches='tight', pad_inches=0, dpi=save_dpi)
/Users/lenasalzmann/dev/dbs-eeg-sync/dbs_eeg_sync/gui.py:534: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  app.processEvents(QtCore.QEventLoop.ProcessEventsFlag.AllEvents if hasattr(QtCore.QEventLoop, "ProcessEventsFlag") else QtCore.QEventLoop.AllEvents)



✗ Cancelled or error: Manual selection cancelled.


: 